In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
USE DATABASE DEMO_DB;
USE SCHEMA CRM;

In [ ]:
select * from CHAT_TRANSCRIPT limit 10;

In [ ]:
select * from KNOWLEDGE_ARTICLES limit 5;

In [ ]:
-- Create chunks from knowledge article and give it index no 
CREATE OR REPLACE TABLE knowledge_chunks AS
SELECT
  q.articlenumber,                 -- add these in
  q.title,
  q.summary,
  f.index + 1       AS chunk_id,      
  f.value::string   AS chunk_text
FROM KNOWLEDGE_ARTICLES AS q,
LATERAL FLATTEN(
  INPUT => SNOWFLAKE.CORTEX.SPLIT_TEXT_RECURSIVE_CHARACTER(
             q.procedure_steps__c,           -- your text column
             'none',             -- format (use 'CHARACTER' for char-based chunks)
             1200,                    -- chunk size
             100                     -- overlap (use 50 if you prefer)
           )
) AS f
WHERE q.procedure_steps__c IS NOT NULL
ORDER BY chunk_id
--limit 10
;

In [ ]:
create or replace table chat_transcripts_kb as select *,
AI_COMPLETE('claude-4-sonnet', 'You are a support search assistant. Given the following customer support transcript, output a contextual search query that will help locate the right knowledge article and action to take. Something like: What do I do when the customer had an order delivered to the wrong address? Do NOT include quotes or extra words. Transcript: ' || transcript) as search_term
from CHAT_TRANSCRIPT;

In [ ]:
select * from CHAT_TRANSCRIPTS_KB limit 5;

In [ ]:
CREATE OR REPLACE TABLE transcripts_with_chunks AS
WITH base AS (
  SELECT
    t.chat_id,
    k.articlenumber,
    k.chunk_id,
    k.chunk_text,
    AI_SIMILARITY(
      k.chunk_text,
      t.search_term,
      OBJECT_CONSTRUCT('model','voyage-multilingual-2')
    ) AS sim
  FROM chat_transcripts_kb t
  JOIN knowledge_chunks k ON TRUE
  WHERE t.search_term IS NOT NULL
),
top5 AS (
  SELECT *
  FROM base
  QUALIFY ROW_NUMBER() OVER (PARTITION BY chat_id ORDER BY sim DESC) <= 5
),
windowed AS (
  SELECT
    t5.chat_id,
    k2.articlenumber,
    k2.chunk_id,
    k2.chunk_text
  FROM top5 t5
  JOIN knowledge_chunks k2
    ON k2.articlenumber = t5.articlenumber
   AND k2.chunk_id BETWEEN t5.chunk_id - 1 AND t5.chunk_id + 1
),
dedup AS (
  SELECT DISTINCT chat_id, articlenumber, chunk_id, chunk_text
  FROM windowed
)
SELECT
  chat_id,
  ARRAY_AGG(
    OBJECT_CONSTRUCT(
      'article',    articlenumber,
      'chunk_id',   chunk_id,
      'Supporting_Knowledge_Details', chunk_text
    )
  ) WITHIN GROUP (ORDER BY articlenumber, chunk_id) AS kb_window_chunks
FROM dedup
GROUP BY chat_id;

In [ ]:
select * from transcripts_with_chunks limit 5;

In [ ]:
create or replace table retail_chat_transcripts_kb as 
select RCT.*,TWC.KB_WINDOW_CHUNKS 
from TRANSCRIPTS_WITH_CHUNKS TWC , CHAT_TRANSCRIPTS_KB RCT
where TWC.chat_id=RCT.chat_id;

In [ ]:
select * from retail_chat_transcripts_kb limit 5;

In [ ]:
CREATE OR REPLACE TABLE QA_SCORING_SUMMARY AS
SELECT
    T.AGENT,
    T.CHAT_ID,
    T.TRANSCRIPT,
    T.KB_WINDOW_CHUNKS,      -- VARIANT or TEXT: concatenated or array of knowledge article texts/URLs
    TO_VARIANT(CASE
        WHEN T.TRANSCRIPT IS NULL OR TRIM(T.TRANSCRIPT) = ''
        THEN NULL
        ELSE AI_COMPLETE(
        model => 'claude-4-sonnet',
        prompt =>
'## ROLE
You are a senior QA analyst evaluating call/chat interactions for Acme Corp. Score the interaction strictly against the rubric provided. Use ONLY the supplied sources (transcript, Salesforce case & item records, and knowledge articles). If a required detail is not present in the supplied sources, mark "insufficient_evidence": true for that element and score it 0 (do NOT guess).

## CRITICAL JSON VALIDATION REQUIREMENTS
- YOU MUST COPY THE EXACT JSON TEMPLATE BELOW AND ONLY CHANGE THE VALUES
- NEVER omit any field from any element
- EVERY element MUST have: "name", "score", "max_score", "insufficient_evidence", "rationale", "evidence"
- "insufficient_evidence" is MANDATORY - set to true or false for EVERY element
- If you cannot find evidence, set "insufficient_evidence": true and "score": 0

## OUTPUT CONTRACT (CRITICAL)
Return ONE valid JSON object and nothing else. Follow the schema exactly. Every element MUST include a brief "rationale" and an "evidence" array of verbatim snippets (≤25 words each) citing the exact source used: "transcript", "sf_case", "sf_items", "kb". If no evidence exists to support the scoring element, use an empty array and set "insufficient_evidence": true.


## RUBRIC & ELEMENT DEFINITIONS

### 3) Policy/Process Education
> Sources: transcript + knowledge articles
> Goal: Educator explained the resolution offered to the Guest and provided accurate education around Acme Corp policy for the resolution offered.
> Scoring:
- 2 pt: Educator explained the resolution offered accurately according to all available sources
- 1 pt: Educator explained the resolution offered, BUT some of the information was either missing or inaccurate
- 0 pt: Educator did not provide an explanation of the resolution
** Always include a reference to the knowlege article in your rationale and evidence as support for providing the score and describe exactley what the educator should have done if not assigned 2 points, if there is ambiguity but the educator created a good guest experience, you can assign 2. 
***Be extra careful about the root cause. You may see that an order takes 1-2 days to process for shipping but then 2-6 days for actual shipping once processed. Do not get confused in these cases, be careful, identify the root cause, and then reference the appropriate operating guidelines when judging the agent.

### 4) POWER UP: Education on Acme Corp offerings
> Sources: transcript
> Goal: The educator proactively provided the Guest education on Acme Corp offerings to promote Acme Corp community (i.e., beyond education on policy and timelines), such as: Describing membership programs, Highlighting community events, Highlighting available website / app self-service capabilities.
> Scoring:
1 pt - proactively promoted Acme Corp community offerings
0 pt - just stayed on task, did not add any community commentary


### 5) Timeline education 
> Sources: transcript + knowledge articles
> Goal: Educator explained the timelines for the resolution offered to the Guest and provided accurate education around the timelines for the resolution offered
> Scoring:
- 2 pt: Educator provided accurate timelines for the resolution offered according to all available sources
- 1 pt: Educator provided timelines for the resolution offered, but timelines were not accurate or correct
- 0 pt: Educator did not provide education for timelines for the resolution
** If assigning 0 or 1 point, state exactly what the agent should have done to achieve score of 2 in your rationale
**Be extra careful about the root cause. You may see that an order takes 1-2 days to process for shipping but then 2-6 days for actual shipping once processed. Do not get confused in these cases, be careful, identify the root cause, and then reference the appropriate operating guidelines when judging the agent.

###SOFT SKILLS###
> Sources: transcript
### 6) Introduction: Assign 1 pt if the educator introduces themselves by name, else 0.
### 7) Connection: Assign 1 pt if the educator went above-and-beyond empathy/connection (uses guest name, references profile info, references prior info, or small-talk to connect)
### 8) Spelling and grammar: Spelling/grammar/product-name accuracy; no slang/abbr.
  - 2 pts: 0–3 errors
  - 1 pts: 4–6 errors
  - 0 pts: ≥7 errors
### 9) Standard text/Quick replies: Assign 1pt if there are no obvious placeholders from templates left in communication, else 0.
### 10) Tailored acknowledgement: Assign 1pt if educator tailors initial acknowledgement to reason of contact, else 0. Saying I can help is not a tailored acknowledgement. Tailored acknowledgement shows empathy and understanding. Example for a guest unable to cancel an order would be “I’m sorry to hear you can’t cancel your order, let me see how I can help


### 11) Avoid repetition
> Sources: transcript
> Goal: Educator does not repeat previously asked for or discussed information
> Scoring:
- 1 pt: Avoids asking guest to repeat previously provided info; if repeated, acknowledges & thanks
- 0 pt: Asks guests to repeat information and does not acknowledge the inconvenience

### 12) Hold times
> Sources: transcript
> Goal: Enables good guest experience by providing a "by-when" before holds or research time
> Scoring:
- 1 pt: Always provides clear guidance if they need to take time or put the guest on hold and sets proper expectations that they adhere to.
- 0 pt: If they do not provide notice of hold or long pauses or if they do not adhere to expectations. i.e. I will be back in a minute, and does not return for 3 minutes.

### 13) Timely responses
> Sources: transcript timestamps
> Goal: Responds within channel expectations, Phone ≤ 3s, Live chat ≤ 10s, Apple for business ≤ 10s. (If timestamps unavailable, mark insufficient_evidence.)
> Scoring
- 1 pt: If ALWAYS responds according to SLAs, or is clear in justifying why when they are out of SLA
- 0 pt: Misses SLAs and does not provide any justification to the guest

### 14) PCI components validated
> Sources: transcript only.
> Goal: Educator validates ≥3 of 5 PCI components effectively (name, phone, order number, address, email). If there is prior chat history available where a previous agent already captured this information but the agent asked again, assign 0 points. 
> Scoring: 
1 pt - if 3 or more of the 5 components are captured
0 pt - if less than 3 of 5 are captured or the guest was asked repeated information.

### 15) Product-related offering
> Sources: transcript
> Goal: educator proactively drives revenue by making an effort to place another order, recommending products, suggesting DE appointment, explaining Science of Feel etc.
> Scoring:
1 pt - proactively drove incremental revenue in anyway. Offering to re place an order counts.
0 pt - just stayed on task, did not make any new offers

### 16) Resolution Offered
> Sources: transcript + knowledge articles
> Goal: Educator supported the Guests needs by offering a resolution that aligns to Acme Corp policy, training, and/or knowledge base, or providing a resolution that supports Guest experience when necessary for an identified exception in SOP 
> Scoring: 
2 pt - if the resolution aligns to policy/training/KB OR if its justified by exception because the agent pursued the best possible Guest experience. 
1 pt - as long as an appropriate resolution was offered
0 pt - if no resolution or a completely incorrect resolution was provided

### 17) Resolution set-up
> Sources: transcript + knowledge articles + item records
> Goal: Educator accurately created the Reason of Contact in Salesforce
> Scoring:
2 pt - Correct Reason of Contact is listed in Salesforce case
1 pt - If reason of contact is relatively correct
0 pt - If reason of contact is wrong

###System Use Scores###
> Sources: salesforce case record
### 19) Reason of Contact: Assign 1 pt if the reason of contact field correctly reflects the conversation, else 0.
### 20) Case Status: Assign 1pt if case status correct (status is "closed" if no follow-up; "waiting on guest" if guest follow-up needed), else 0.
### 21) Case Subject Line: Assign 1pt if subject field accurately reflects purpose, else 0.
### 22) Guest Profile: Assign 1pt if phone in case matches transcript mention (if phone was mentioned), else 0.
### 23) Guest shipping address: Assign 1pt if address captured correctly IF the guest shared it in the interaction, else 0.
### 24) Supplementary Items: Assign 1pt if item records reflect all actions, questions, and suggestions, else 0.
### 25) Timeline notes: Assign 1pt if case notes accurately capture the call and resolution, else 0.

## INPUTS
You will receive:
- transcript: full text of the interaction
- sf_case: JSON of the Salesforce case record
- kb_articles: concatenated text or array of relevant knowledge articles (policy/SOP)

## DECISION RULES TO REDUCE HALLUCINATIONS
- Cite exact snippets for every non-binary claim.
- If a required datum is not explicitly present in the provided sources, set "insufficient_evidence": true and score 0 for that element; do NOT infer from brand common sense.
- For timestamps/SLAs, only use explicit timing from transcript metadata; if none, mark insufficient_evidence.
- For policy alignment, quote the specific KB passage used.

## REQUIRED JSON TEMPLATE - COPY EXACTLY AND FILL VALUES
IMPORTANT: Copy this exact structure. Do not omit any field. Every element must have "insufficient_evidence": true or false.
{
  "agent_persona_summary": "1–3 sentences on strengths and weaknesses of this interaction backing up the scores provided.",
  "elements": [
        {
          "name": "<insert the name of the scoring element>",
          "element_id": "<insert the ID/number of the corresponding scoring element>",
          "rationale": "<REQUIRED: Brief explanation of scoring decision based on the rubric definition>",
          "score": 0,
          "insufficient_evidence": <boolean true/false value if evidence was present to support the score>,
          "evidence": [<provide the supporting snippet from the transcript, knowledge article, or case value that backs up the scoring rationale>]
        },
        ...
      ]
}
  
## FINAL VALIDATION CHECKLIST - CRITICAL
Before returning your JSON, verify EVERY SINGLE ELEMENT has these 6 fields:
1. "name" (string)
2. "element_id" (string)
3. "score" (integer) 
4. "insufficient_evidence" (boolean - true or false)
5. "rationale" (string)
6. "evidence" (array of strings)

MISSING ANY OF THESE FIELDS WILL CAUSE AN ERROR. Check every element in every category.

## SOURCE DATA TO USE FOR THE ASSESSMENT
# TRANSCRIPT
<<TRANSCRIPT_START>>
' || COALESCE(T.transcript, 'No transcript available') || '
<<TRANSCRIPT_END>>



# KNOWLEDGE ARTICLES (TEXT OR JSON)
<<KB_ARTICLES_START>>
' || to_json(T.KB_WINDOW_CHUNKS)::STRING || '
<<KB_ARTICLES_END>>',
  response_format => {
    'type': 'json',
    'schema': {
      'type': 'object',
      'properties': {
        'agent_persona_summary': { 'type': 'string' },
        'elements': {
          'type': 'array',
          'items': {
            'type': 'object',
            'properties': {
              'name': { 'type': 'string' },
              'element_id': { 'type': 'string' },
              'rationale': { 'type': 'string' },
              'score': { 'type': 'integer' },
              'insufficient_evidence': { 'type': 'boolean' },
              'evidence': {
                'type': 'array',
                'items': { 'type': 'string' }
              }
            },
            'required': [
              'name',
              'element_id',
              'rationale',
              'score',
              'insufficient_evidence',
              'evidence'
            ],
            'additionalProperties': false
          }
        }
      },
      'required': ['agent_persona_summary', 'elements'],
      'additionalProperties': false
    }
  },
        model_parameters => {
            'temperature': 0
        }
        )
    END ) AS qa_score
FROM
    retail_chat_transcripts_kb AS T;

In [ ]:
select * from QA_SCORING_SUMMARY limit 5;

In [ ]:
select * from HUMAN_QM_SCORES limit 5;

In [ ]:
create or replace view SCORING_VIEW
as
WITH llm AS (
  SELECT
      q.chat_id,
      q.agent,
      q.transcript,
      q.KB_WINDOW_CHUNKS,
      q.qa_score:"agent_persona_summary"::string AS agent_persona_summary,
      e.value:"element_id"::string               AS element_id,
      e.value:"name"::string                     AS element_name,        -- col1
      e.value:"score"::number                    AS llm_score,           -- col2
      e.value:"rationale"::string                AS element_rationale,
      e.value:"insufficient_evidence"::boolean   AS insufficient_evidence,
      e.value:"evidence"                         AS evidence             -- array/VARIANT
  FROM qa_scoring_summary q,
       LATERAL FLATTEN(input => q.qa_score:"elements") e
)
SELECT distinct
    llm.element_name                             AS element_name,
    llm.element_id                               AS element_id,    
    llm.llm_score                                AS llm_score,
    TRY_TO_NUMBER(h.score)                       AS human_score,
    (llm.llm_score = TRY_TO_NUMBER(h.score))     AS scores_match,
     TRY_TO_NUMBER(h.possible_score)             AS possible_max_score,
    llm.insufficient_evidence                    AS insufficient_evidence,
    llm.element_rationale                        AS rationale,
    llm.evidence                                 AS evidence,
    llm.agent                                    AS agent,
    llm.chat_id                                  AS chat_id,
    llm.agent_persona_summary                    AS agent_persona_summary,
    llm.transcript                               AS transcript,
    llm.KB_WINDOW_CHUNKS
FROM llm
LEFT JOIN HUMAN_QM_SCORES h
  ON h.chat_id = llm.chat_id
  and h.element_id = llm.element_id
ORDER BY llm.chat_id, llm.element_id ASC;


In [ ]:
select * from SCORING_VIEW limit 10;

## Additional analyis

In [ ]:
--CREATE OR REPLACE TABLE insights AS
SELECT
    T.AGENT,
    T.CHAT_ID,
    T.TRANSCRIPT,
    TO_VARIANT(CASE
        WHEN T.TRANSCRIPT IS NULL OR TRIM(T.TRANSCRIPT) = ''
        THEN NULL
        ELSE AI_COMPLETE(
        model => 'claude-4-sonnet',
        prompt =>
'## ROLE
You are a senior QA analyst evaluating call/chat interactions for Acme Corp. Given the call transcript please output the following details in JSON format, no preceeding or trailing characters in the {} json response.

## REQUIRED JSON TEMPLATE - COPY EXACTLY AND FILL VALUES
IMPORTANT: Copy this exact structure. Do not omit any field.
{
  "llm_reason_of_contact": <2-3 word reason for call>,
  "guest_dropped": <boolean value representing if the guest dropped from the conversation. You will see evidence of the educator asking if guest is still there>,
  "guest_dropped_detail": <1 sentence explanation of how and ideally why the guest dropped the conversation. If guest did not drop, leave blank>,
  "guest_frustrated_w_bot": <boolean value representing whether the guest was frustrated with the bot before being transferred to a live educator>,
  "guest_frustrated_w_bot_detail": <1-2 sentences explaining why the bot was furstrating the customer. If guest was not frustrated, leave blank.>
}

# SOURCE DATA TO USE FOR THE ASSESSMENT
## TRANSCRIPT
<<TRANSCRIPT_START>>
' || COALESCE(T.transcript, 'No transcript available') || '
<<TRANSCRIPT_END>>
',
  response_format => {
    'type': 'json',
    'schema': {
      'type': 'object',
      'properties': {
        'llm_reason_of_contact': {'type': 'string'},
        'guest_dropped' : {'type': 'boolean'},
        'guest_dropped_detail': {'type': 'string'},
        'guest_frustrated_w_bot': {'type': 'boolean'},
        'guest_frustrated_w_bot_detail': {'type': 'string'}
      },
      'required': ['llm_reason_of_contact', 'guest_dropped', 'guest_frustrated_w_bot', 'guest_frustrated_w_bot_detail','guest_dropped_detail' ],
      'additionalProperties': false
    }
  },
        model_parameters => {
            'temperature': 0
        }
        )
    END ) AS insights_output
FROM
    retail_chat_transcripts_kb AS T
LIMIT 1;

In [ ]:
CREATE OR REPLACE VIEW insights_flattened AS
SELECT
    AGENT,
    CHAT_ID,
    TRANSCRIPT,
    SF_CASE,
    REASON_OF_CONTACT__C,
    STATUS,
    SUBJECT,
    insights_output:"llm_reason_of_contact"::STRING AS llm_reason_of_contact,
    insights_output:"guest_dropped"::BOOLEAN AS guest_dropped,
    insights_output:"guest_dropped_detail"::STRING AS guest_dropped_detail,
    insights_output:"guest_frustrated_w_bot"::BOOLEAN AS guest_frustrated_w_bot,
    insights_output:"guest_frustrated_w_bot_detail"::STRING AS guest_frustrated_w_bot_detail
FROM insights;

In [ ]:
select * from InSIGHTS_FLATTENED ;